## Data Ingestion Yield

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
import mysql.connector
import geopandas as gpd


In [2]:
df = pd.read_csv('32100002.csv')

C:\ProgramData\Anaconda3\envs\FieldCrops\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
CARUID = df.loc[ : , 'GEO']
CARUID

0                                    Québec
1                                    Québec
2                                    Québec
3                                    Québec
4                                    Québec
                        ...                
317891    Peace River, 8 - British Columbia
317892    Peace River, 8 - British Columbia
317893    Peace River, 8 - British Columbia
317894    Peace River, 8 - British Columbia
317895    Peace River, 8 - British Columbia
Name: GEO, Length: 317896, dtype: object

In [4]:
for i in range(0, len(CARUID)):

    location = CARUID.values[i]

    if(location.find(' - ') != -1):
        location = location.split(' - ')
        province = location[1].strip()
        region = location[0].split()[-1].strip()
        if(province == 'Manitoba'):
            province = '46'
        elif(province.find('Saskatchewan') != -1):
            province = '47'
        elif(province == 'Alberta'):
            province = '48'
        elif(province == 'British Columbia'):
            province = '59'
        elif(province == 'Ontario'):
            province = '35'
        elif(province == 'Qu' + u'\u00E9' + 'bec'):
            province = '24'
        else:
            print(province)
            province = '00'
        if(len(region) == 1):
            region = '0' + region
        CARUID.values[i] = int(province + region)
    else:
        CARUID.values[i] = 0

In [5]:
df['Harvest disposition'].unique()

array(['Seeded area (acres)', 'Seeded area (hectares)',
       'Harvested area (acres)', 'Harvested area (hectares)',
       'Average yield (bushels per acre)',
       'Average yield (kilograms per hectare)', 'Production (bushels)',
       'Production (metric tonnes)', 'Average yield (pounds per acre)',
       'Production (pounds)'], dtype=object)

#### df_production

In [6]:
df_production = df[df['Harvest disposition'] == 'Production (metric tonnes)']
df_production = df_production[~(df_production['VALUE'].isna())]
df_production = df_production[df_production['GEO'] != 0]
df_production = df_production.loc[ : , ['REF_DATE', 'GEO', 'Type of crop', 'VALUE']]
df_production = df_production.reset_index().drop(columns = ['index'])
df_production

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,102200.0
1,1976,3501,Corn for grain,2196000.0
2,1976,3501,Oats,118200.0
3,1976,3501,"Wheat, all",494500.0
4,1976,3502,Barley,173500.0
...,...,...,...,...
22722,2020,5908,Canola,51800.0
22723,2020,5908,Oats,50500.0
22724,2020,5908,"Peas, dry",24100.0
22725,2020,5908,"Rye, fall remaining",0.0


In [7]:
print(df_production['REF_DATE'].unique())
print(df_production['GEO'].unique())
print(df_production['Type of crop'].unique())

[1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020]
[3501 3502 3503 3504 3505 4701 4702 4703 4704 4705 4706 4707 4708 4709
 4710 4711 4712 4713 4714 4715 4716 4717 4718 4719 4720 4810 4820 4830
 4840 4841 4850 4860 4870 4601 4602 4603 4604 4605 4606 4607 4608 4609
 4610 4611 4612 5901 5908 2401 2402 2403 2404 2405 2406 2407 2408 2409
 2410 2411]
['Barley' 'Corn for grain' 'Oats' 'Wheat, all' 'Canola' 'Flaxseed'
 'Wheat, durum' 'Wheat, spring' 'Rye, fall remaining' 'Rye, spring'
 'Wheat, winter remaining' 'Lentils' 'Mustard seed' 'Peas, dry'
 'Sunflower seed' 'Canary seed' 'Soybeans' 'Rye, all' 'Chick peas'
 'Triticale']


In [8]:
df_SK = df_production[(df_production['GEO'] > 4700) & (df_production['GEO'] < 4800)]
print(df_SK.shape)
df_QC = df_production[(df_production['GEO'] > 2400) & (df_production['GEO'] < 2500)]
print(df_QC.shape)

(11215, 4)
(660, 4)


In [9]:
# Small area data regions in SK are mapped to census agricultural regions as below.

df_SK_4701 = df_SK[(df_SK['GEO'] == 4701) | (df_SK['GEO'] == 4705) | (df_SK['GEO'] == 4709) | (df_SK['GEO'] == 4710)]
df_SK_4702 = df_SK[(df_SK['GEO'] == 4702) | (df_SK['GEO'] == 4706)]
df_SK_4703 = df_SK[(df_SK['GEO'] == 4703) | (df_SK['GEO'] == 4704) | (df_SK['GEO'] == 4707) | (df_SK['GEO'] == 4708)]
df_SK_4704 = df_SK[(df_SK['GEO'] == 4711) | (df_SK['GEO'] == 4712) | (df_SK['GEO'] == 4713)]
df_SK_4705 = df_SK[(df_SK['GEO'] == 4714)]
df_SK_4706 = df_SK[(df_SK['GEO'] == 4715) | (df_SK['GEO'] == 4716) | (df_SK['GEO'] == 4717)]
df_SK_4707 = df_SK[(df_SK['GEO'] == 4718)]

In [10]:
df_SK_4701 = df_SK_4701.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4702 = df_SK_4702.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4703 = df_SK_4703.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4704 = df_SK_4704.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4705 = df_SK_4705.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4706 = df_SK_4706.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4707 = df_SK_4707.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()

In [11]:
df_SK_4701['GEO'] = 4701
df_SK_4702['GEO'] = 4702
df_SK_4703['GEO'] = 4703
df_SK_4704['GEO'] = 4704
df_SK_4705['GEO'] = 4705
df_SK_4706['GEO'] = 4706
df_SK_4707['GEO'] = 4707

In [12]:
df_SK = pd.concat([df_SK_4701, df_SK_4702, df_SK_4703, df_SK_4704, df_SK_4705, df_SK_4706, df_SK_4707], axis = 0)
df_SK.reset_index().drop(columns = ['index'])

,REF_DATE,Type of crop,VALUE,GEO
0,1976,Barley,143600.0,4701
1,1976,Canola,3600.0,4701
2,1976,Flaxseed,11400.0,4701
3,1976,Oats,157500.0,4701
4,1976,"Wheat, all",2324000.0,4701
...,...,...,...,...
4221,2016,Triticale,0.0,4707
4222,2016,"Wheat, all",571800.0,4707
4223,2016,"Wheat, durum",22200.0,4707
4224,2016,"Wheat, spring",549600.0,4707


In [13]:
df_production = df_production[(df_production['GEO'] < 4700) | (df_production['GEO'] > 4800)]
df_production.shape

(11512, 4)

In [14]:
df_production = pd.concat([df_production, df_SK], axis = 0)
df_production.shape

(15738, 4)

In [15]:
# Small area data regions in QC are mapped to census agricultural regions as below.

df_QC = df_QC.reset_index().drop(columns = ['index'])

for i in range(0, len(df_QC)):
    if(df_QC.iloc[i].GEO == 2404):
        df_QC.loc[i:i, 'GEO'] = 2405
        continue
    elif(df_QC.iloc[i].GEO == 2405):
        df_QC.loc[i:i, 'GEO'] = 2406
        continue
    elif(df_QC.iloc[i].GEO == 2406):
        df_QC.loc[i:i, 'GEO'] = 2408
        continue
    elif(df_QC.iloc[i].GEO == 2407):
        df_QC.loc[i:i, 'GEO'] = 2410
        continue
    elif(df_QC.iloc[i].GEO == 2408):
        df_QC.loc[i:i, 'GEO'] = 2412
        continue
    elif(df_QC.iloc[i].GEO == 2411):
        df_QC.loc[i:i, 'GEO'] = 2414
    else:
        continue
        
df_QC_2413 = df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)]
print(df_QC_2413.shape)
df_QC_2413 = df_QC_2413.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_QC_2413['GEO'] = 2413
print(df_QC_2413.shape)

(158, 4)
(70, 4)


In [16]:
df_QC.drop(df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)].index, inplace=True)
df_QC.shape

(502, 4)

In [17]:
df_QC = pd.concat([df_QC, df_QC_2413], axis = 0).reset_index().drop(columns = ['index'])
df_QC.shape

(572, 4)

In [18]:
df_production = df_production[(df_production['GEO'] < 2400) | (df_production['GEO'] > 2500)]
df_production.shape

(15078, 4)

In [19]:
df_production = pd.concat([df_production, df_QC], axis = 0)
df_production.shape

(15650, 4)

In [20]:
df_production.reset_index().drop(columns = ['index'])

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,102200.0
1,1976,3501,Corn for grain,2196000.0
2,1976,3501,Oats,118200.0
3,1976,3501,"Wheat, all",494500.0
4,1976,3502,Barley,173500.0
...,...,...,...,...
15645,2020,2413,Barley,26900.0
15646,2020,2413,Corn for grain,2039200.0
15647,2020,2413,Oats,38300.0
15648,2020,2413,Soybeans,588700.0


#### df_seeded

In [21]:
df_seeded = df[df['Harvest disposition'] == 'Seeded area (hectares)']
df_seeded = df_seeded[~(df_seeded['VALUE'].isna())]
df_seeded = df_seeded[df_seeded['GEO'] != 0]
df_seeded.drop(df_seeded[df_seeded['Type of crop'] == 'Summerfallow'].index, inplace = True)
df_seeded = df_seeded.loc[:,['REF_DATE', 'GEO', 'Type of crop', 'VALUE']]
df_seeded = df_seeded.reset_index().drop(columns = ['index'])
df_seeded

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,43400.0
1,1976,3501,Corn for grain,390800.0
2,1976,3501,Oats,65000.0
3,1976,3501,"Wheat, all",149300.0
4,1976,3502,Barley,69700.0
...,...,...,...,...
22725,2020,5908,Canola,35300.0
22726,2020,5908,Oats,23700.0
22727,2020,5908,"Peas, dry",23200.0
22728,2020,5908,"Rye, fall remaining",0.0


In [22]:
print(df_seeded['REF_DATE'].unique())
print(df_seeded['GEO'].unique())
print(df_seeded['Type of crop'].unique())

[1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020]
[3501 3502 3503 3504 3505 4701 4702 4703 4704 4705 4706 4707 4708 4709
 4710 4711 4712 4713 4714 4715 4716 4717 4718 4719 4720 4810 4820 4830
 4840 4841 4850 4860 4870 4601 4602 4603 4604 4605 4606 4607 4608 4609
 4610 4611 4612 5901 5908 2401 2402 2403 2404 2405 2406 2407 2408 2409
 2410 2411]
['Barley' 'Corn for grain' 'Oats' 'Wheat, all' 'Canola' 'Flaxseed'
 'Wheat, durum' 'Wheat, spring' 'Rye, fall remaining' 'Rye, spring'
 'Wheat, winter remaining' 'Lentils' 'Mustard seed' 'Peas, dry'
 'Sunflower seed' 'Canary seed' 'Soybeans' 'Rye, all' 'Chick peas'
 'Triticale']


In [23]:
df_SK = df_seeded[(df_seeded['GEO'] > 4700) & (df_seeded['GEO'] < 4800)]
print(df_SK.shape)
df_QC = df_seeded[(df_seeded['GEO'] > 2400) & (df_seeded['GEO'] < 2500)]
print(df_QC.shape)

(11215, 4)
(660, 4)


In [24]:
# Small area data regions in SK are mapped to census agricultural regions as below.

df_SK_4701 = df_SK[(df_SK['GEO'] == 4701) | (df_SK['GEO'] == 4705) | (df_SK['GEO'] == 4709) | (df_SK['GEO'] == 4710)]
df_SK_4702 = df_SK[(df_SK['GEO'] == 4702) | (df_SK['GEO'] == 4706)]
df_SK_4703 = df_SK[(df_SK['GEO'] == 4703) | (df_SK['GEO'] == 4704) | (df_SK['GEO'] == 4707) | (df_SK['GEO'] == 4708)]
df_SK_4704 = df_SK[(df_SK['GEO'] == 4711) | (df_SK['GEO'] == 4712) | (df_SK['GEO'] == 4713)]
df_SK_4705 = df_SK[(df_SK['GEO'] == 4714)]
df_SK_4706 = df_SK[(df_SK['GEO'] == 4715) | (df_SK['GEO'] == 4716) | (df_SK['GEO'] == 4717)]
df_SK_4707 = df_SK[(df_SK['GEO'] == 4718)]

In [25]:
df_SK_4701 = df_SK_4701.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4702 = df_SK_4702.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4703 = df_SK_4703.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4704 = df_SK_4704.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4705 = df_SK_4705.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4706 = df_SK_4706.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4707 = df_SK_4707.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()

In [26]:
df_SK_4701['GEO'] = 4701
df_SK_4702['GEO'] = 4702
df_SK_4703['GEO'] = 4703
df_SK_4704['GEO'] = 4704
df_SK_4705['GEO'] = 4705
df_SK_4706['GEO'] = 4706
df_SK_4707['GEO'] = 4707

In [27]:
df_SK = pd.concat([df_SK_4701, df_SK_4702, df_SK_4703, df_SK_4704, df_SK_4705, df_SK_4706, df_SK_4707], axis = 0)
df_SK.reset_index().drop(columns = ['index'])

,REF_DATE,Type of crop,VALUE,GEO
0,1976,Barley,67500.0,4701
1,1976,Canola,3400.0,4701
2,1976,Flaxseed,14500.0,4701
3,1976,Oats,100800.0,4701
4,1976,"Wheat, all",1239600.0,4701
...,...,...,...,...
4221,2016,Triticale,0.0,4707
4222,2016,"Wheat, all",193600.0,4707
4223,2016,"Wheat, durum",7500.0,4707
4224,2016,"Wheat, spring",186500.0,4707


In [28]:
df_seeded = df_seeded[(df_seeded['GEO'] < 4700) | (df_seeded['GEO'] > 4800)]
df_seeded.shape

(11515, 4)

In [29]:
df_seeded = pd.concat([df_seeded, df_SK], axis = 0)
df_seeded.shape

(15741, 4)

In [30]:
# Small area data regions in QC are mapped to census agricultural regions as below.

df_QC = df_QC.reset_index().drop(columns = ['index'])

for i in range(0, len(df_QC)):
    if(df_QC.iloc[i].GEO == 2404):
        df_QC.loc[i:i, 'GEO'] = 2405
        continue
    elif(df_QC.iloc[i].GEO == 2405):
        df_QC.loc[i:i, 'GEO'] = 2406
        continue
    elif(df_QC.iloc[i].GEO == 2406):
        df_QC.loc[i:i, 'GEO'] = 2408
        continue
    elif(df_QC.iloc[i].GEO == 2407):
        df_QC.loc[i:i, 'GEO'] = 2410
        continue
    elif(df_QC.iloc[i].GEO == 2408):
        df_QC.loc[i:i, 'GEO'] = 2412
        continue
    elif(df_QC.iloc[i].GEO == 2411):
        df_QC.loc[i:i, 'GEO'] = 2414
    else:
        continue
        
df_QC_2413 = df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)]
print(df_QC_2413.shape)
df_QC_2413 = df_QC_2413.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_QC_2413['GEO'] = 2413
print(df_QC_2413.shape)

(158, 4)
(70, 4)


In [31]:
df_QC.drop(df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)].index, inplace=True)
df_QC.shape

(502, 4)

In [32]:
df_QC = pd.concat([df_QC, df_QC_2413], axis = 0).reset_index().drop(columns = ['index'])
df_QC.shape

(572, 4)

In [33]:
df_seeded = df_seeded[(df_seeded['GEO'] < 2400) | (df_seeded['GEO'] > 2500)]
df_seeded.shape

(15081, 4)

In [34]:
df_seeded = pd.concat([df_seeded, df_QC], axis = 0)
df_seeded.shape

(15653, 4)

In [35]:
df_seeded.reset_index().drop(columns = ['index'])

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,43400.0
1,1976,3501,Corn for grain,390800.0
2,1976,3501,Oats,65000.0
3,1976,3501,"Wheat, all",149300.0
4,1976,3502,Barley,69700.0
...,...,...,...,...
15648,2020,2413,Barley,9300.0
15649,2020,2413,Corn for grain,218800.0
15650,2020,2413,Oats,17000.0
15651,2020,2413,Soybeans,169500.0


#### df_harvested

In [36]:
df_harvested = df[df['Harvest disposition'] == 'Harvested area (hectares)']
df_harvested = df_harvested[~(df_harvested['VALUE'].isna())]
df_harvested = df_harvested[df_harvested['GEO'] != 0]
df_harvested = df_harvested.loc[ : , ['REF_DATE', 'GEO', 'Type of crop', 'VALUE']]
df_harvested = df_harvested.reset_index().drop(columns = ['index'])
df_harvested

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,43400.0
1,1976,3501,Corn for grain,390800.0
2,1976,3501,Oats,65000.0
3,1976,3501,"Wheat, all",149300.0
4,1976,3502,Barley,69700.0
...,...,...,...,...
22722,2020,5908,Canola,31400.0
22723,2020,5908,Oats,17500.0
22724,2020,5908,"Peas, dry",19000.0
22725,2020,5908,"Rye, fall remaining",0.0


In [37]:
print(df_harvested['REF_DATE'].unique())
print(df_harvested['GEO'].unique())
print(df_harvested['Type of crop'].unique())

[1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020]
[3501 3502 3503 3504 3505 4701 4702 4703 4704 4705 4706 4707 4708 4709
 4710 4711 4712 4713 4714 4715 4716 4717 4718 4719 4720 4810 4820 4830
 4840 4841 4850 4860 4870 4601 4602 4603 4604 4605 4606 4607 4608 4609
 4610 4611 4612 5901 5908 2401 2402 2403 2404 2405 2406 2407 2408 2409
 2410 2411]
['Barley' 'Corn for grain' 'Oats' 'Wheat, all' 'Canola' 'Flaxseed'
 'Wheat, durum' 'Wheat, spring' 'Rye, fall remaining' 'Rye, spring'
 'Wheat, winter remaining' 'Lentils' 'Mustard seed' 'Peas, dry'
 'Sunflower seed' 'Canary seed' 'Soybeans' 'Rye, all' 'Chick peas'
 'Triticale']


In [38]:
df_SK = df_harvested[(df_harvested['GEO'] > 4700) & (df_harvested['GEO'] < 4800)]
print(df_SK.shape)
df_QC = df_harvested[(df_harvested['GEO'] > 2400) & (df_harvested['GEO'] < 2500)]
print(df_QC.shape)

(11215, 4)
(660, 4)


In [39]:
# Small area data regions in SK are mapped to census agricultural regions as below.

df_SK_4701 = df_SK[(df_SK['GEO'] == 4701) | (df_SK['GEO'] == 4705) | (df_SK['GEO'] == 4709) | (df_SK['GEO'] == 4710)]
df_SK_4702 = df_SK[(df_SK['GEO'] == 4702) | (df_SK['GEO'] == 4706)]
df_SK_4703 = df_SK[(df_SK['GEO'] == 4703) | (df_SK['GEO'] == 4704) | (df_SK['GEO'] == 4707) | (df_SK['GEO'] == 4708)]
df_SK_4704 = df_SK[(df_SK['GEO'] == 4711) | (df_SK['GEO'] == 4712) | (df_SK['GEO'] == 4713)]
df_SK_4705 = df_SK[(df_SK['GEO'] == 4714)]
df_SK_4706 = df_SK[(df_SK['GEO'] == 4715) | (df_SK['GEO'] == 4716) | (df_SK['GEO'] == 4717)]
df_SK_4707 = df_SK[(df_SK['GEO'] == 4718)]

In [40]:
df_SK_4701 = df_SK_4701.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4702 = df_SK_4702.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4703 = df_SK_4703.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4704 = df_SK_4704.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4705 = df_SK_4705.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4706 = df_SK_4706.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_SK_4707 = df_SK_4707.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()

In [41]:
df_SK_4701['GEO'] = 4701
df_SK_4702['GEO'] = 4702
df_SK_4703['GEO'] = 4703
df_SK_4704['GEO'] = 4704
df_SK_4705['GEO'] = 4705
df_SK_4706['GEO'] = 4706
df_SK_4707['GEO'] = 4707

In [42]:
df_SK = pd.concat([df_SK_4701, df_SK_4702, df_SK_4703, df_SK_4704, df_SK_4705, df_SK_4706, df_SK_4707], axis = 0)
df_SK.reset_index().drop(columns = ['index'])

,REF_DATE,Type of crop,VALUE,GEO
0,1976,Barley,64800.0,4701
1,1976,Canola,3400.0,4701
2,1976,Flaxseed,14500.0,4701
3,1976,Oats,80900.0,4701
4,1976,"Wheat, all",1239600.0,4701
...,...,...,...,...
4221,2016,Triticale,0.0,4707
4222,2016,"Wheat, all",184200.0,4707
4223,2016,"Wheat, durum",6700.0,4707
4224,2016,"Wheat, spring",177500.0,4707


In [43]:
df_harvested = df_harvested[(df_harvested['GEO'] < 4700) | (df_harvested['GEO'] > 4800)]
df_harvested.shape

(11512, 4)

In [44]:
df_harvested = pd.concat([df_harvested, df_SK], axis = 0)
df_harvested.shape

(15738, 4)

In [45]:
# Small area data regions in QC are mapped to census agricultural regions as below.

df_QC = df_QC.reset_index().drop(columns = ['index'])

for i in range(0, len(df_QC)):
    if(df_QC.iloc[i].GEO == 2404):
        df_QC.loc[i:i, 'GEO'] = 2405
        continue
    elif(df_QC.iloc[i].GEO == 2405):
        df_QC.loc[i:i, 'GEO'] = 2406
        continue
    elif(df_QC.iloc[i].GEO == 2406):
        df_QC.loc[i:i, 'GEO'] = 2408
        continue
    elif(df_QC.iloc[i].GEO == 2407):
        df_QC.loc[i:i, 'GEO'] = 2410
        continue
    elif(df_QC.iloc[i].GEO == 2408):
        df_QC.loc[i:i, 'GEO'] = 2412
        continue
    elif(df_QC.iloc[i].GEO == 2411):
        df_QC.loc[i:i, 'GEO'] = 2414
    else:
        continue
        
df_QC_2413 = df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)]
print(df_QC_2413.shape)
df_QC_2413 = df_QC_2413.groupby(['REF_DATE', 'Type of crop']).sum('VALUE').reset_index()
df_QC_2413['GEO'] = 2413
print(df_QC_2413.shape)

(158, 4)
(70, 4)


In [46]:
df_QC.drop(df_QC[(df_QC['GEO'] == 2409) | (df_QC['GEO'] == 2410)].index, inplace=True)
df_QC.shape

(502, 4)

In [47]:
df_QC = pd.concat([df_QC, df_QC_2413], axis = 0).reset_index().drop(columns = ['index'])
df_QC.shape

(572, 4)

In [48]:
df_harvested = df_harvested[(df_harvested['GEO'] < 2400) | (df_harvested['GEO'] > 2500)]
df_harvested.shape

(15078, 4)

In [49]:
df_harvested = pd.concat([df_harvested, df_QC], axis = 0)
df_harvested.shape

(15650, 4)

In [50]:
df_harvested.reset_index().drop(columns = ['index'])

,REF_DATE,GEO,Type of crop,VALUE
0,1976,3501,Barley,43400.0
1,1976,3501,Corn for grain,390800.0
2,1976,3501,Oats,65000.0
3,1976,3501,"Wheat, all",149300.0
4,1976,3502,Barley,69700.0
...,...,...,...,...
15645,2020,2413,Barley,9200.0
15646,2020,2413,Corn for grain,218300.0
15647,2020,2413,Oats,16500.0
15648,2020,2413,Soybeans,169300.0


#### df_yield

In [51]:
df_production.rename(columns = { 'Type of crop' : 'Type_of_crop' }, inplace = True)
df_seeded.rename(columns = { 'Type of crop' : 'Type_of_crop' }, inplace = True)
df_harvested.rename(columns = { 'Type of crop' : 'Type_of_crop' }, inplace = True)

In [52]:
df_yield = df_production.copy()
df_yield['Seeded'] = np.NaN
df_yield['Harvested'] = np.NaN
df_yield

,REF_DATE,GEO,Type_of_crop,VALUE,Seeded,Harvested
0,1976,3501,Barley,102200.0,NaN,NaN
1,1976,3501,Corn for grain,2196000.0,NaN,NaN
2,1976,3501,Oats,118200.0,NaN,NaN
3,1976,3501,"Wheat, all",494500.0,NaN,NaN
4,1976,3502,Barley,173500.0,NaN,NaN
...,...,...,...,...,...,...
567,2020,2413,Barley,26900.0,NaN,NaN
568,2020,2413,Corn for grain,2039200.0,NaN,NaN
569,2020,2413,Oats,38300.0,NaN,NaN
570,2020,2413,Soybeans,588700.0,NaN,NaN


In [53]:
df_yield.rename(columns = { 'VALUE' : 'Production'}, inplace = True)
df_yield.columns

Index(['REF_DATE', 'GEO', 'Type_of_crop', 'Production', 'Seeded', 'Harvested'], dtype='object')

In [54]:
counter_found = 0
counter_not_found = 0
counter_duplicate = 0

for i in range (0, len(df_seeded)):
    record = df_seeded.iloc[i]
    index = df_yield[(df_yield['REF_DATE'] == record.REF_DATE) & (df_yield['GEO'] == record.GEO) & \
                 (df_yield['Type_of_crop'] == record.Type_of_crop)].index
    if len(index) == 1:
        counter_found += 1
        #print(record.VALUE)
        df_yield.loc[index, 'Seeded'] = record.VALUE
    if len(index) == 0:
        counter_not_found += 1
    if len(index) > 1:
        counter_duplicate += 1
    

print('found:', counter_found)
print('not found:', counter_not_found)
print('duplicate:', counter_duplicate)

found: 15649
not found: 4
duplicate: 0


In [55]:
counter_found = 0
counter_not_found = 0
counter_duplicate = 0

for i in range (0, len(df_harvested)):
    record = df_harvested.iloc[i]
    index = df_yield[(df_yield['REF_DATE'] == record.REF_DATE) & (df_yield['GEO'] == record.GEO) & \
                 (df_yield['Type_of_crop'] == record.Type_of_crop)].index
    if len(index) == 1:
        counter_found += 1
        #print(record.VALUE)
        df_yield.loc[index, 'Harvested'] = record.VALUE
    if len(index) == 0:
        counter_not_found += 1
    if len(index) > 1:
        counter_duplicate += 1
    

print('found:', counter_found)
print('not found:', counter_not_found)
print('duplicate:', counter_duplicate)

found: 15650
not found: 0
duplicate: 0


In [59]:
df_yield.dropna(inplace = True)

In [56]:
df_yield.tail(100)

,REF_DATE,GEO,Type_of_crop,Production,Seeded,Harvested
472,2020,2403,Barley,6800.0,3000.0,2900.0
473,2020,2403,Corn for grain,165800.0,19800.0,19600.0
474,2020,2403,Oats,12800.0,7500.0,6800.0
475,2020,2403,Soybeans,97800.0,33300.0,33000.0
476,2020,2403,"Wheat, all",15700.0,9600.0,9300.0
477,2020,2405,Barley,4100.0,1500.0,1500.0
478,2020,2405,Corn for grain,95900.0,10900.0,10900.0
479,2020,2405,Oats,3900.0,1900.0,1700.0
480,2020,2405,Soybeans,27900.0,9000.0,8900.0
481,2020,2405,"Wheat, all",9300.0,5100.0,4900.0


#### INSERT

In [60]:
connection = mysql.connector.connect(user='root', password='your_password', host='127.0.0.1', database='field_crops')
cursor = connection.cursor()

add_yield = ("INSERT INTO yield "
           "(CARUID, year, crop, mass, seeded, harvested) "
           "VALUES (%s, %s, %s, %s, %s, %s)")

for i in range(0, len(df_yield)):

    record = df_yield.iloc[i]
    
    CARUID = int(record.GEO)
    year = int(record.REF_DATE)
    crop = record.Type_of_crop
    mass = int(record.Production)
    seeded = int(record.Seeded)
    harvested = int(record.Harvested)
    
    
    data_yield = (CARUID, year, crop, mass, seeded, harvested)
    
    cursor.execute(add_yield, data_yield)
    connection.commit()


cursor.close()
connection.close()